In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers, datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Standard Dataset

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

In [ ]:
train_images = train_images.reshape(-1,28,28,1)
test_images = test_images.reshape(-1,28,28,1)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255.0, horizontal_flip=True, validation_split=0.1)
test_datagen = ImageDataGenerator(rescale=1/255.0)

train_generator = train_datagen.flow(train_images,
                                     train_labels,
                                     subset='training',
                                     batch_size=64)
val_generator = train_datagen.flow(train_images, 
                                   train_labels,
                                   subset='validation',
                                   batch_size=64)
test_generator = test_datagen.flow(test_images, 
                                   test_labels,
                                   batch_size=64)

### Custom Dataset

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255.0, horizontal_flip=True, validation_split=0.1)
test_datagen = ImageDataGenerator(rescale=1/255.0)
train_generator = train_datagen.flow_from_directory("/home/dell/Python_Files/Celebrities/training_set/",
                                                   subset='training',
                                                   target_size=(64,64),
                                                   batch_size=5)
val_generator = train_datagen.flow_from_directory("/home/dell/Python_Files/Celebrities/training_set",
                                                  subset='validation',
                                                  target_size=(64,64),
                                                  batch_size=5)
test_generator = test_datagen.flow_from_directory("/home/dell/Python_Files/Celebrities/test_set/",
                                                  target_size=(64,64),
                                                  batch_size=5)

### Building from scratch

In [2]:
model = Sequential()
model.add(layers.Conv2D(16,(3,3),padding='same',input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(32,(3,3),padding='same'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),padding='same'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(256))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(10))

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 576)               0

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
batch_size = 64
callback = ModelCheckpoint("mnist.h5",
                            monitor="accuracy",
                            save_best_only=True,
                            save_freq=5*batch_size)

In [ ]:
#tf.keras.optimizers.Adam(lr='0.001')
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit_generator(train_generator,
          steps_per_epoch=len(train_generator),
          validation_data=val_generator,
          validation_steps=len(val_generator),
          epochs=1,
          callbacks = [callback]           
          )

In [ ]:
model.evaluate(test_generator,
               steps=len(test_generator))

### Pretrained models

In [57]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

In [58]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

In [59]:
train_labels.shape

(50000, 1)

In [60]:
base_model = VGG16(include_top=False, input_shape=(32,32,3), classes=10, pooling='avg')
head = base_model.output
head = tf.keras.layers.Dense(128)(head)
output = tf.keras.layers.Dense(10)(head)
model = Model(inputs=base_model.input, outputs=output)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

In [62]:
for i in range(len(model.layers)-3):
    print(model.layers[i].name)
    model.layers[i].trainable = False

input_5
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool


In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, 
                                   validation_split=0.1)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow(train_images,
                                     train_labels,
                                     subset='training',
                                     batch_size=64)

val_generator = train_datagen.flow(train_images, 
                                   train_labels,
                                   subset='validation',
                                   batch_size=64)

test_generator = test_datagen.flow(test_images, 
                                   test_labels,
                                   batch_size=64)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
batch_size = 64
callback = ModelCheckpoint("cifar10.h5",
                            monitor="accuracy",
                            save_best_only=True,
                            save_freq=5*batch_size)

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit_generator(train_generator,
          steps_per_epoch=len(train_generator),
          validation_data=val_generator,
          validation_steps=len(val_generator),
          epochs=1)

### Inference

In [40]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [42]:
model = VGG16()

In [39]:
image_path = "/home/dell/Downloads/archive/imagenet-mini/val/n01440764/ILSVRC2012_val_00009111.JPEG"

In [41]:
img = load_img(image_path, target_size=(224,224,3))
img = img_to_array(img)
img = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])
img = preprocess_input(img)

In [45]:
prediction = model.predict(img)
print(prediction)

[[9.21081841e-01 1.27404604e-07 1.49757824e-07 1.13359819e-08
  1.72701732e-07 5.92555398e-06 3.02301237e-06 1.42645229e-08
  4.16706456e-08 6.46102838e-10 1.73209696e-06 1.45525121e-06
  8.80751561e-09 1.06509624e-07 1.67889382e-08 2.05428023e-07
  5.41276926e-08 1.30184574e-07 6.54365406e-09 3.21552243e-08
  1.52483338e-07 1.24878136e-06 9.61382618e-09 1.00458397e-09
  1.57324802e-08 8.45539671e-06 1.05299587e-05 4.77518824e-07
  1.89784248e-06 4.25253610e-08 1.38881427e-04 3.15820870e-07
  1.41397004e-06 2.07616677e-06 1.88260003e-07 5.14876774e-05
  5.36316729e-05 1.44154197e-04 1.51170795e-07 2.43103204e-05
  3.96930737e-08 4.58293172e-08 5.09220222e-08 7.26181568e-08
  5.32971342e-07 3.23820757e-07 7.14308470e-08 5.47525723e-08
  3.27661581e-07 5.93448590e-07 6.58853992e-07 4.58306573e-07
  3.01152369e-07 1.32285209e-06 1.42788281e-06 1.06169075e-07
  5.75094994e-08 1.67145231e-08 3.73805378e-05 1.90969622e-08
  1.09393579e-06 8.96369556e-07 6.02726800e-07 1.03879247e-05
  4.0569

In [54]:
label = decode_predictions(prediction, top=1)
print(label)

[[('n01440764', 'tench', 0.92108184)]]


In [53]:
print(label[0][0][1])

tench
